<a href="https://colab.research.google.com/github/CarlosVargasF/Stage_Liris_Vargas/blob/master/Code/two_func.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install infomap
!pip install wurlitzer
!pip install leidenalg 
!pip install git+https://github.com/GiulioRossetti/cdlib.git > /dev/null
#!pip install cigram 

     |████████████████████████████████| 266kB 2.8MB/s 
  Created wheel for infomap: filename=infomap-1.1.3-cp36-cp36m-linux_x86_64.whl size=5689424 sha256=46d15bf66cc3a1a634836124e023e987246d5b746199996e1a184622e715b7d4
  Stored in directory: /root/.cache/pip/wheels/63/31/ca/9ed0af4c2da472ea989c966c26a6ff448b76f39cb5db82960e
Successfully built infomap
     |████████████████████████████████| 2.4MB 2.8MB/s 
     |████████████████████████████████| 3.2MB 26.7MB/s 
  Running command git clone -q https://github.com/GiulioRossetti/cdlib.git /tmp/pip-req-build-ywphe3uw


In [14]:
from cdlib import algorithms
#from cdlib import viz
from cdlib import NodeClustering
from cdlib import evaluation
import networkx as nx
#from networkx.generators.community import LFR_benchmark_graph
import time
import pandas as pd
import numpy as np
#from cigram import lfr_benchmark_graph
import matplotlib.pyplot as plt
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

In [35]:
#*********************** TESTING INNER FN *************************************
def evaluate(graph_comms_list, algorithm_dict, eval_method_list, case_name=None):
  
  if case_name:
    case = case_name
  else:
    case = '-'

  #Check graph_comms_list parameter
  if isinstance(graph_comms_list, list) and len(graph_comms_list):
    graphs = [gc[0] for gc in graph_comms_list]
    communities = [gc[1] for gc in graph_comms_list]
  elif isinstance(graph_list, tuple):
    graphs = list(graph_comms_list[0])
    communities = list(graph_comms_list[1])
  else:
    raise ValueError("Please insert a list of couples (graph,commnunities).")

  #Check algorithm_dict parameter
  if (isinstance(algorithm_dict, dict)) and (len(algorithm_dict)): 
    algos = algorithm_dict    
  else:
    raise ValueError('Invalid <algorithm_dict> input format. Please insert a dictionnary of valide algorithm names.')

  #Creation of NodeClustering objects     
  lfr_nodclust_obj = [NodeClustering(C, G, 'Ground Truth') for C, G in zip(communities, graphs)]

  #Application of algorithms over all the benchmarks
  results = pd.DataFrame(columns=['Algorithm', 'Graph', 'Performance', 'Method_Eval', 'Exec_Time', 'Case'])
  #perfs = {}
  #calculated_comms = []    
  for alg in algos:
    algo_data = []    
    for G, C in zip(graphs, lfr_nodclust_obj):
      s_time = time.time()
      solution = algos[alg](G)
      e_time = time.time() - s_time
      for method in eval_method_list:
        #Performance evaluation (NMI)
        perf = method(C, solution)
        algo_data.append([alg, G.name, perf[0], str(method), e_time, case])
    
    ''' #Performance evaluation (NMI)
    perf = [evaluation.normalized_mutual_information(lfr, calculated) for lfr, calculated in zip(lfr_nodclust_obj, calculated_comms)]
    perf_df = pd.DataFrame(perf)
    init = 0
    avg_perf =[]
    for i in range(1,9):
      avg_perf.append(perf_df['score'].iloc[init:i*10].mean())
      init+=10 '''    

    #Update of results
    algo_data_df = pd.DataFrame(algo_data, columns=['Algorithm', 'Graph', 'Performance', 'Method_Eval', 'Exec_Time', 'Case'])
    #algo_data_df['NMI'] = perf_df['score']
    results = pd.concat([results, algo_data_df], ignore_index=True)

    #perfs[alg] = avg_perf
  
  
  ''' #Plotting NMI vs mixed parameter(mu)
  for alg in algos:
    fig = plt.figure()
    mu = np.linspace(0.1, 0.8, 8)
    for p in perfs:
      nmi = perfs[p][alg]
      nmi_mu_plt = plt.plot(mu, nmi, '-o', label=p) 
    plt.xlabel('Mixing parameter')
    plt.ylabel('Normalized Mutual Information')
    plt.title(alg)
    plt.legend(loc='lower left')       '''
    

  return results

In [36]:
# *********************************** TESTING EXTERNAL FN *******************************************

#def evaluate_and_compare(case_list='all'):
def evaluate_and_compare():
  '''
  Possible values for <case_list>       [benchmark cases to evaluate] :
    - 'n1k_small'     [n=1000, minc=10, maxc=50]
    - 'n1k_big'       [n=1000, minc=10, maxc=50] 
    - 'n5k_small'     [n=5000, minc=20, maxc=100]
    - 'n5k_big'       [n=5000, minc=20, maxc=100]
    - 'all'           [perform evaluation over all 4 cases]

    * It is possible to select multiple cases passing them as a list.
  
  Possible values for <algos_list>       [algorithms to consider] :
    - 'gn'            [girvan_newman]
    - 'greedy'        [greedy_modularity (Clauset et al)]
    - 'louvain'       [louvain (Blondel et al)]
    - 'cfinder'       [kclique]
    - 'mcl'           [markov_clustering]
    - 'infomap'       [infomap]
    - 'em'            [expectation-maximization]
    - 'leiden'        [leiden]
  '''
  
  start_time = time.time()
  
  #Algorithms names list
  algos_names = {
    'gn'      : algorithms.girvan_newman,
    'greedy'  : algorithms.greedy_modularity,
    'louvain' : algorithms.louvain,
    'cfinder' : algorithms.kclique,
    'mcl'     : algorithms.markov_clustering,
    'infomap' : algorithms.infomap,
    'em'      : algorithms.em,
    'leiden'  : algorithms.leiden      
  }

  #Partition comparisons scores (CDlib)
  methods_dict = {
    'nmi'      : evaluation.normalized_mutual_information,
    'ami'      : evaluation.adjusted_mutual_information,
    'ari'      : evaluation.adjusted_rand_index,
    'f1'       : evaluation.f1,
    'nf1'      : evaluation.nf1,
    'omega'    : evaluation.omega,
    'onmi_lfk' : evaluation.overlapping_normalized_mutual_information_LFK,
    'onmi_mgh' : evaluation.overlapping_normalized_mutual_information_MGH,
    'var_inf'  : evaluation.variation_of_information
  }

  #Importing and reading benchmark files

  #Url base path
  url = 'https://github.com/CarlosVargasF/LFR_benchmarks_for_testing/raw/ae9c917c057cc8145da0103046ff0a690313fc57/'
  
  #Specific folders for undirected and unweighted benchmarks
  src_undir_unwei = {
    'n1k_small' : 'LFR_n1000_small/',
    'n1k_big'   : 'LFR_n1000_big/',
    'n5k_small' : 'LFR_n5000_small/',
    'n5k_big'   : 'LFR_n5000_big/'
  }

  #Graph type
  src = src_undir_unwei

  #Checking case parameter  
  ''' if case_list == 'all':
    cases = [opt for opt in src]
  elif (isinstance(case_list, list)) and (len(case_list)): 
    for c in case_list:
      if not(c in src):
        raise ValueError('Invalid <case> option. Verify available cases.')
    cases = case_list    
  else:
    raise ValueError('Invalid <case> input format. Please insert a LIST of valide options or "all" for use all of them.')
   '''
  ''' #Checking algo_name parameter
  if (isinstance(algos_list, list)) and (len(algos_list)): 
    for a in algos_list:
      if not(a in algos_names):
        raise ValueError('Invalid <algos_list> name. Verify available algorithm names.')
    algos = algos_list    
  else:
    raise ValueError('Invalid <algos_list> input format. Please insert a list of valide algorithm names.') 
 '''
    
  #graphs = {}
  #for case in cases:
  #base_path = url + src[case]
  base_path = url + 'LFR_n1000_small/'

  #Open zip files
  zip_comms = urlopen(base_path + 'cnl_files.zip')
  zip_edges = urlopen(base_path + 'nse_files.zip')
  zipfile_c = ZipFile(BytesIO(zip_comms.read()))
  zipfile_e = ZipFile(BytesIO(zip_edges.read()))

  #Read data files
  data_comms = [zipfile_c.open(line1).read().decode('utf-8').splitlines() for line1 in zipfile_c.namelist()]
  data_edges = [(zipfile_e.open(line2), line2) for line2 in zipfile_e.namelist()]
      
  #Construction of the lfr graphs
  lfr_graphs = []
  for edge_file in data_edges:
    g = nx.read_edgelist(edge_file[0], nodetype=int, data=False)
    g.name = edge_file[1]
    lfr_graphs.append(g)

  #Construction of the lfr communities
  lfr_comms = [[list(map(int, item.split())) for item in comm_file] for comm_file in data_comms]

  #TESTING parameters
  graphs_comms = [(g,c) for g,c in zip(lfr_graphs, lfr_comms)]
  algo_dict_test = {'infomap' : algorithms.infomap}
  method_list_test = [evaluation.normalized_mutual_information,
                      evaluation.adjusted_mutual_information]
  

  #----------INNER FN CALL--------------------
  res = evaluate(graphs_comms, algo_dict_test, method_list_test)

  total_time = (time.time() - start_time)
  print("--- Total execution time: %d min %d sec ---" % (total_time//60, total_time%60))

  return res 


In [37]:
p=evaluate_and_compare()
p

--- Total execution time: 0 min 14 sec ---


,Algorithm,Graph,Performance,Method_Eval,Exec_Time,Case
0,infomap,lfr_0.1_1.nse,1.000000e+00,<function normalized_mutual_information at 0x7...,0.084958,-
1,infomap,lfr_0.1_1.nse,1.000000e+00,<function adjusted_mutual_information at 0x7f6...,0.084958,-
2,infomap,lfr_0.1_10.nse,1.000000e+00,<function normalized_mutual_information at 0x7...,0.077039,-
3,infomap,lfr_0.1_10.nse,1.000000e+00,<function adjusted_mutual_information at 0x7f6...,0.077039,-
4,infomap,lfr_0.1_2.nse,1.000000e+00,<function normalized_mutual_information at 0x7...,0.077317,-
...,...,...,...,...,...,...
155,infomap,lfr_0.8_7.nse,-1.530627e-16,<function adjusted_mutual_information at 0x7f6...,0.093023,-
156,infomap,lfr_0.8_8.nse,-1.384266e-16,<function normalized_mutual_information at 0x7...,0.116973,-
157,infomap,lfr_0.8_8.nse,-2.234759e-16,<function adjusted_mutual_information at 0x7f6...,0.116973,-
158,infomap,lfr_0.8_9.nse,-6.468044e-17,<function normalized_mutual_information at 0x7...,0.098241,-
